<a href="https://colab.research.google.com/github/Adetayo047/Abosedeuk/blob/main/Intent_Recognition_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT RE-WRITTEN

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00


In [3]:
# Reading Data from csvfiles
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/intent Recognition/Pidgin Intent.xlsx')
df.head()

,text,intent
0,listen to westbam alumb allergic on google music,PlayMusic
1,add step to me for 50 clásicos playlist,AddToPlaylist
2,I give this current textbook a rating value of...,RateBook
3,play the song small robin redbreast,PlayMusic
4,abeg add iris dement to my playlist this na se...,AddToPlaylist


In [5]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split

df = pd.read_excel('/content/drive/MyDrive/intent Recognition/Pidgin Intent.xlsx')

# Split the data into training and testing sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Label encode the intent labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['intent_enc'] = le.fit_transform(train['intent'])
test['intent_enc'] = le.transform(test['intent'])

# Load the BERT tokenizer and encode the texts

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['intent_enc'] = le.fit_transform(train['intent'])
test['intent_enc'] = le.fit_transform(test['intent'])


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(train.text), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(list(test.text), truncation=True, padding=True, return_tensors='tf')

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train.intent_enc))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test.intent_enc))

# Fine-tune the pre-trained BERT model for intent recognition
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=train['intent_enc'].nunique())
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
model.fit(train_dataset.shuffle(len(train.text)).batch(32), epochs=3, batch_size=32)

# Evaluate the model on the test set
results = model.evaluate(test_dataset.batch(32))
print("Test accuracy:", results[1])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
25/25 [==============================] - 77s 409ms/step - loss: 1.7969 - accuracy: 0.3850
Epoch 2/3
25/25 [==============================] - 7s 271ms/step - loss: 1.2140 - accuracy: 0.7950
Epoch 3/3
7/7 [==============================] - 4s 86ms/step - loss: 0.3788 - accuracy: 0.9550
Test accuracy: 0.9549999833106995


In [6]:
model.save_pretrained('bertmodelintent')

# Optionally, you can save the tokenizer as well
tokenizer.save_pretrained('berttokenizer')

('berttokenizer/tokenizer_config.json',
 'berttokenizer/special_tokens_map.json',
 'berttokenizer/vocab.txt',
 'berttokenizer/added_tokens.json')

In [7]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load the model from the saved directory
model = TFBertForSequenceClassification.from_pretrained('bertmodelintent')

# Optionally, load the tokenizer as well
tokenizer = BertTokenizer.from_pretrained('berttokenizer')

# Function to predict intent from a given text
def predict_intent(text):
    # Tokenize the text
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='tf')

    # Get the model's prediction
    predictions = model(inputs)[0]
    predicted_label_id = tf.argmax(predictions, axis=1).numpy()[0]

    return le.inverse_transform([predicted_label_id])[0]

# Example usage


Some layers from the model checkpoint at bertmodelintent were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bertmodelintent.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [11]:
text_to_predict = "abeg you fit add am to my music"
predicted_intent = predict_intent(text_to_predict)
print("Predicted intent:", predicted_intent)

Predicted intent: AddToPlaylist


In [ ]:
test